In [1]:
import json

# Đọc dữ liệu từ file
train_data = []
with open("duration_training_dataset.txt", "r", encoding="utf-8") as f:  # ← đổi tên file cho đúng với file bạn đang dùng
    for line in f:
        if line.strip():  # Bỏ qua dòng trống
            train_data.append(json.loads(line.strip()))

print(f"Số lượng mẫu: {len(train_data)}")
print("Ví dụ một mẫu đầu tiên:")
print(json.dumps(train_data[0], indent=2, ensure_ascii=False))


Số lượng mẫu: 1490
Ví dụ một mẫu đầu tiên:
{
  "context": "Trong một lớp học, các học sinh đang học về các chủ đề khác nhau. Một số em rất chăm chỉ và thường xuyên hoàn thành bài tập về nhà, trong khi những em khác lại bỏ lỡ nhiều bài. Cô giáo luôn cố gắng khuyến khích cả lớp bằng cách tổ chức các buổi thảo luận thú vị.",
  "labels": [
    "yes",
    "no",
    "yes",
    "no"
  ],
  "options": [
    "1 tuần",
    "5 năm",
    "10 ngày",
    "10 giây"
  ],
  "qid": 650,
  "question": "Mất bao lâu để hoàn thành tất cả bài tập về nhà của lớp học?"
}


In [2]:
from collections import Counter

label_counter = Counter()
option_lengths = []

for sample in train_data:
    label_counter.update(sample["labels"])
    option_lengths.append(len(sample["options"]))

print("Tổng số nhãn:")
print(label_counter)
print(f"Số option trung bình mỗi câu hỏi: {sum(option_lengths) / len(option_lengths):.2f}")


Tổng số nhãn:
Counter({'no': 3003, 'yes': 2957})
Số option trung bình mỗi câu hỏi: 4.00


In [4]:

# BƯỚC 2: Rule-based labeling
def rule_based_predict(sample):
    context = sample["context"].lower()
    question = sample["question"].lower()
    options = sample["options"]

    short_units = ["giây", "phút", "giờ"]
    medium_units = ["ngày", "tuần"]
    long_units = ["tháng", "năm"]

    short_context = ["biểu diễn", "sửa", "trình diễn", "hòa nhạc", "vẽ", "chơi", "làm bài", "bài tập"]
    long_context = ["chuẩn bị", "triển lãm", "phát triển", "hoàn thành", "tổ chức"]

    predicted_labels = []

    for opt in options:
        label = "no"  # default

        # Check type of time unit in option
        if any(unit in opt for unit in short_units):
            if any(word in context for word in short_context):
                label = "yes"
        elif any(unit in opt for unit in medium_units):
            if any(word in context for word in long_context + short_context):
                label = "yes"
        elif any(unit in opt for unit in long_units):
            if any(word in context for word in long_context):
                label = "yes"

        predicted_labels.append(label)

    return predicted_labels

# BƯỚC 3: Đánh giá
from sklearn.metrics import classification_report

y_true = []
y_pred = []

for sample in train_data:
    pred = rule_based_predict(sample)
    gold = sample["labels"]
    y_true.extend(gold)
    y_pred.extend(pred)

print(classification_report(y_true, y_pred, target_names=["no", "yes"]))


              precision    recall  f1-score   support

          no       0.51      0.83      0.63      3003
         yes       0.51      0.18      0.26      2957

    accuracy                           0.51      5960
   macro avg       0.51      0.50      0.45      5960
weighted avg       0.51      0.51      0.45      5960



In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=2)


/home/chuanbico/miniconda3/envs/vlsp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/chuanbico/miniconda3/envs/vlsp/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias',

In [5]:
import json

examples = []
with open("duration_training_dataset.txt", "r", encoding="utf-8") as f:
    for line in f:
        sample = json.loads(line.strip())
        for opt, label in zip(sample["options"], sample["labels"]):
            text = sample["context"] + " " + sample["question"] + " " + opt
            label_id = 1 if label == "yes" else 0
            examples.append({
                "text": text,
                "label": label_id
            })

print(f"Tổng số sample: {len(examples)}")

Tổng số sample: 5960


In [6]:
from datasets import Dataset

dataset = Dataset.from_list(examples)

def preprocess(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

encoded_dataset = dataset.map(preprocess, batched=True)
encoded_dataset = encoded_dataset.train_test_split(test_size=0.2)

Map: 100%|██████████| 5960/5960 [00:00<00:00, 6818.10 examples/s]


In [7]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./phobert_duration_qa",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
)

trainer.train()

/home/chuanbico/miniconda3/envs/vlsp/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 10/5960 [00:06<35:53,  2.76it/s] 

{'loss': 0.7066, 'learning_rate': 4.9916107382550336e-05, 'epoch': 0.02}


  0%|          | 20/5960 [00:09<31:59,  3.09it/s]

{'loss': 0.7102, 'learning_rate': 4.983221476510067e-05, 'epoch': 0.03}


  1%|          | 30/5960 [00:13<31:43,  3.12it/s]

...

                                                   
100%|██████████| 5960/5960 [36:46<00:00,  2.87it/s]

{'eval_loss': 0.666321337223053, 'eval_runtime': 14.9707, 'eval_samples_per_second': 79.622, 'eval_steps_per_second': 9.953, 'epoch': 10.0}


100%|██████████| 5960/5960 [36:53<00:00,  2.69it/s]

{'train_runtime': 2213.1349, 'train_samples_per_second': 21.544, 'train_steps_per_second': 2.693, 'train_loss': 0.2339770257262038, 'epoch': 10.0}


TrainOutput(global_step=5960, training_loss=0.2339770257262038, metrics={'train_runtime': 2213.1349, 'train_samples_per_second': 21.544, 'train_steps_per_second': 2.693, 'train_loss': 0.2339770257262038, 'epoch': 10.0})

In [8]:
import numpy as np
from sklearn.metrics import classification_report

predictions = trainer.predict(encoded_dataset["test"])
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = predictions.label_ids

print(classification_report(y_true, y_pred, target_names=["no", "yes"]))


100%|██████████| 149/149 [00:13<00:00, 10.70it/s]

              precision    recall  f1-score   support

          no       0.91      0.91      0.91       611
         yes       0.90      0.90      0.90       581

    accuracy                           0.90      1192
   macro avg       0.90      0.90      0.90      1192
weighted avg       0.90      0.90      0.90      1192



In [9]:
trainer.save_model("phobert_duration_model")


In [10]:
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load model đã fine-tune
model_path = "./phobert_duration_model"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()  # chuyển sang chế độ dự đoán

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Hàm chuyển nhãn
def id_to_label(pred):
    return "yes" if pred == 1 else "no"

# Hàm dự đoán nhãn cho từng option
def predict_labels(context, question, options):
    labels = []
    for option in options:
        text = context + " " + question + " " + option
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            pred = torch.argmax(logits, dim=1).item()
            labels.append(id_to_label(pred))
    return labels

# Đọc file test
test_data = []
with open("public_test.txt", "r", encoding="utf-8") as f:
    for line in f:
        sample = json.loads(line.strip())
        test_data.append(sample)

# Dự đoán và ghi kết quả
with open("submission.txt", "w", encoding="utf-8") as f:
    for sample in test_data:
        predicted_labels = predict_labels(sample["context"], sample["question"], sample["options"])
        sample["labels"] = predicted_labels
        f.write(json.dumps(sample, ensure_ascii=False) + "\n")

print("✅ Dự đoán hoàn tất! Kết quả đã lưu vào file: submission.txt")


✅ Dự đoán hoàn tất! Kết quả đã lưu vào file: submission.txt


In [27]:
import random

# Đọc toàn bộ submission
with open("submission.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

# Chọn ngẫu nhiên 3 dòng
samples = random.sample(lines, k=min(3, len(lines)))

# In nội dung từng dòng
for idx, line in enumerate(samples, 1):
    sample = json.loads(line)
    print(f"\n🧩 Sample {idx} — QID: {sample['qid']}")
    print(f"📘 Context: {sample['context']}")
    print(f"❓ Question: {sample['question']}")
    print("🧠 Options + Labels:")
    for opt, label in zip(sample["options"], sample["labels"]):
        print(f"   - {opt:<15} → {label}")


🧩 Sample 1 — QID: 116
📘 Context: Một nhóm phóng viên đang điều tra một vụ bê bối lớn trong ngành công nghiệp thực phẩm. Họ đã thu thập rất nhiều tài liệu và chứng cứ từ các nguồn khác nhau để làm rõ sự thật.
❓ Question: Mất bao lâu để một nhóm phóng viên hoàn thành điều tra vụ bê bối trong ngành công nghiệp thực phẩm?
🧠 Options + Labels:
   - 3 tuần          → yes
   - 6 tuần          → yes
   - 1 tháng         → yes
   - 12 giờ          → yes

🧩 Sample 2 — QID: 77
📘 Context: Trong một ngôi làng nhỏ, có một nhóm cư dân quyết định tổ chức một lễ hội để kỷ niệm sự đoàn kết của họ. Họ đã lên kế hoạch cho các hoạt động như thi nấu ăn, biểu diễn văn nghệ và nhiều trò chơi dân gian khác.
❓ Question: Mất bao lâu để tổ chức lễ hội này?
🧠 Options + Labels:
   - 3 tuần          → yes
   - 1 ngày          → yes
   - 2 tháng         → yes
   - 5 tuần          → yes

🧩 Sample 3 — QID: 108
📘 Context: Trong một buổi họp báo, phóng viên đã đặt câu hỏi về một sự kiện quan trọng sắp diễn ra, nơi mà nhữ

In [8]:
import json
import re
import numpy as np
from sklearn.metrics import classification_report

def load_labels(path):
    """
    Đọc file JSON (mỗi dòng một object hoặc nối tiếp) và trả về dict: {qid: [0/1,...]}
    với 0 = "no", 1 = "yes".
    """
    text = open(path, 'r', encoding='utf-8').read()
    raw_objs = re.findall(r'\{.*?\}', text, re.DOTALL)
    data = {}
    for obj in raw_objs:
        entry = json.loads(obj)
        # Chuyển "yes"/"no" thành 1/0
        bin_labels = [1 if lab=="yes" else 0 for lab in entry['labels']]
        data[entry['qid']] = bin_labels
    return data

# Đường dẫn tới hai file
gt_path = 'public_test_full_labeled.json'
sub_path = 'submission.json'

# Load nhãn
gt = load_labels(gt_path)
sub = load_labels(sub_path)

# Xây dựng y_true, y_pred theo thứ tự qid nhất định (vd sort theo qid)
y_true, y_pred = [], []
for qid in sorted(gt.keys()):
    true = gt[qid]
    pred = sub.get(qid, [0]*len(true))
    y_true.extend(true)
    y_pred.extend(pred)

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# In classification report
print(classification_report(y_true, y_pred, target_names=["no", "yes"]))


              precision    recall  f1-score   support

          no       0.26      0.48      0.34       206
         yes       0.85      0.69      0.76       906

    accuracy                           0.65      1112
   macro avg       0.56      0.59      0.55      1112
weighted avg       0.74      0.65      0.69      1112



In [9]:
import json
import re
from collections import Counter

def load_json_objects(path):
    text = open(path, 'r', encoding='utf-8').read()
    # \{.*?\} với DOTALL để bắt cả xuống dòng
    raw_objs = re.findall(r'\{.*?\}', text, re.DOTALL)
    return [json.loads(obj) for obj in raw_objs]

# 1. Load ground truth và submission
gt_list = load_json_objects('public_test_full_labeled.json')
sub_list = load_json_objects('submission.json')

# Convert sang dict {qid: labels}
gt = {e['qid']: e['labels'] for e in gt_list}
sub = {e['qid']: e['labels'] for e in sub_list}

# 2. So sánh và build comparison_results
comparison = []
for qid, true_labels in gt.items():
    pred_labels = sub.get(qid, [""] * len(true_labels))
    correct_flags = [t == p for t, p in zip(true_labels, pred_labels)]
    comparison.append({
        "qid": qid,
        "true_labels": true_labels,
        "pred_labels": pred_labels,
        "correct": correct_flags
    })

with open('comparison_results.json', 'w', encoding='utf-8') as f:
    json.dump(comparison, f, ensure_ascii=False, indent=2)

# 3. Tính precision, recall, F1 cho 'yes' và 'no'
counters = {
    'yes': Counter({'tp':0,'fp':0,'fn':0}),
    'no':  Counter({'tp':0,'fp':0,'fn':0})
}

for entry in comparison:
    for true, pred in zip(entry['true_labels'], entry['pred_labels']):
        # yes
        if pred == 'yes':
            counters['yes']['tp' if true=='yes' else 'fp'] += 1
        elif true == 'yes':
            counters['yes']['fn'] += 1
        # no
        if pred == 'no':
            counters['no']['tp' if true=='no' else 'fp'] += 1
        elif true == 'no':
            counters['no']['fn'] += 1

metrics = {}
for lbl, cnt in counters.items():
    tp, fp, fn = cnt['tp'], cnt['fp'], cnt['fn']
    precision = tp/(tp+fp) if tp+fp else 0
    recall    = tp/(tp+fn) if tp+fn else 0
    f1        = 2*precision*recall/(precision+recall) if precision+recall else 0
    metrics[lbl] = {
        'precision': precision,
        'recall':    recall,
        'f1':        f1
    }

# 4. In kết quả
for lbl, m in metrics.items():
    print(f"Label '{lbl}': "
          f"Precision: {m['precision']:.2%}, "
          f"Recall: {m['recall']:.2%}, "
          f"F1: {m['f1']:.2%}")


Label 'yes': Precision: 85.40%, Recall: 69.09%, F1: 76.39%
Label 'no': Precision: 26.12%, Recall: 48.06%, F1: 33.85%
